# Processing Recipes into Data 

### Takes recipes from both BigOven and Spoonacular API and puts them into data formats useable in modeling. Needs to be rerun after pulling more recipes 

##### Includes all deduping



In [1]:
import pandas as pd
import json
from pathlib import Path
import string
import re
import pickle
from itertools import chain
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from itertools import chain
from collections import Counter
import nltk
from pathlib import Path

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/bridgitboulahanis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOinstructions = []
BOtitle = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOinstructions.append(r.get('Instructions'))
        BOtitle.append(r.get('Title'))
        BOids.append(r.get('RecipeID'))
        if BOing is not None:
            BOingredients.append([lemmatizer.lemmatize(re.sub(r'[^a-z ]', '', i['Name'].lower().strip()))
                                  for i in BOing if i is not None and i['Name'] is not None])
            


/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-181-96-1158777.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-164-249-1283792.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-205-249-1797822.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/

In [3]:
print('Current recipe count: ' + str(len(BOinstructions)))
print('Current list of ingredient count: ' + str(len(BOingredients)))
numing = []
numing.extend([len(r) for r in BOingredients])
print('Number of ingredients: ' + str(sum(numing)))
aa = list(chain.from_iterable(BOingredients))
print('Unique ingredients: ' + str(len(Counter(aa).keys())))


Current recipe count: 62160
Current list of ingredient count: 62160
Number of ingredients: 691631
Unique ingredients: 127577


In [4]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle 
# this will be our classification data

SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SAingID = []
SAmeta = []
SAmeasure = []

seenID = set()
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAnoise.extend(i.strip() for i in ing['metaInformation'])
                if ing['id'] in seenID:
                    continue
                seenID.add(ing['id']) 
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                SAingID.append(ing['id'])
                SAmeasure.append(ing['measures']['us']['unitLong'])
                SAmeta.extend([ing['meta']])

                    





In [5]:
#grab all the ingrdients from spoonacular and put in dataframe
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass, 'meta': SAmeta, 'measure': SAmeasure,  'id': SAingID})


In [6]:
#some words I noticed should get added to SAnoise
wta = ['ground', 'taste', 'ounce', 'large']
SAnoise.extend(wta)
SAnoise = set(SAnoise)


In [17]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 
#here we are removing meta words and stop words
stop_words = set(stopwords.words('english')) 


def removenoise(input_ing, posdict):
    assert type(input_ing) == list, type(input_ing)
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        for it in splitit:
            if it not in SAnoise and it not in stop_words:
                tag = nltk.pos_tag([it])
                if tag[0][1] in ['JJ', 'NN', 'NNP', 'VBN']: #check if it is a adj, noun, proper noun
                    checked.append(it)
                  
                elif tag[0][1] in ['JJR', 'JJS', 'NNS', 'NNPS']: # check if adj, noun, propr noun, but comparitive or plural
                    checked.append(lemmatizer.lemmatize(it)) #lemmatize these words
                    
                else: #otherwise let's throw those words into a dict just to make sure we aren't stripping any real data
                    if tag[0][1] not in posdict:
                        posdict[tag[0][1]] = set([it])
                    else:
                        posdict[tag[0][1]].add(it)
        if len(checked) > 0:
            noise_free_ing.append(' '.join(list(filter(None, checked))))
    return noise_free_ing
        


In [18]:
#call the dedupe functions
nning = []
posdict = {}
for i in BOingredients:
    nning.append(removenoise(i, posdict))



In [30]:
print('Unique ingredients: ' + str(len(Counter(list(chain.from_iterable(BOingredients))).keys())))
print('Unique deduped ingredients: ' + str(len(Counter(list(chain.from_iterable(nning))).keys())))


Unique ingredients: 127577
Unique deduped ingredients: 79200


In [31]:
# classify ingredients into aisles
#Note! This is a very slow loop (~40 mins)
whichaisle = {}
seen = set()
notin = []
countclassified = 0
unclassified = 0
for i in nning:
    for j in i:
        if j in seen:
            continue
        seen.add(j) 
        highest = process.extractOne(j,classifying['ingredient']) #using FW to get the closest ingredient match
        a = classifying.index[classifying['ingredient'] == highest[0]] #pulling out the index of that ingredient in aisle df
        if highest[1] > 80: #set minimum bar for ingredient similarity
            whichaisle[j] = classifying['aisle'][a].tolist() # put in aisle classification
            countclassified += 1
        else:
            whichaisle[j]  = None
            notin.append(j)
            unclassified += 1
        

In [34]:
#output the data 
pickle.dump(nning,open("eatlocal/generated_data/cleaned_ingredients.pkl","wb"))
pickle.dump(whichaisle,open("eatlocal/generated_data/ingredient_aisle.pkl","wb"))
pickle.dump(BOinstructions, open("eatlocal/generated_data/instructions.pkl", "wb"))
pickle.dump(BOtitle, open("eatlocal/generated_data/recipetitle.pkl", "wb"))
